In [1]:
from nltk.tokenize import RegexpTokenizer
import cPickle as pickle
import string
import os

%matplotlib inline
import numpy as np
import random
import time

import logging
from logging import info

from multiprocessing import Pool as ThreadPool
import itertools

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
root_location = "/big/s/shalaby/"
exports_location = root_location + "exported_data/"

#training_file = root_location + 'docs_output_training_validation_documents_' + str(SAMPLE_RATIO)
training_file = root_location + 'docs_output.json'

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
# training_docs_list_file = exports_location + "training_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
# validation_docs_list_file = exports_location + "validation_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"

In [4]:
training_docs_additional_list_file = exports_location + "balanced_additional_training_docs_list.pkl"
validation_docs_additional_list_file = exports_location + "balanced_additional_validation_docs_list.pkl"

In [5]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))

additional_training_docs_list = pickle.load(open(training_docs_additional_list_file))
additional_validation_docs_list = pickle.load(open(validation_docs_additional_list_file))

CPU times: user 25.8 s, sys: 880 ms, total: 26.7 s
Wall time: 26.7 s


In [22]:
len(additional_training_docs_list)

783

In [6]:
def stemtokenizer(text):
    """ MAIN FUNCTION to get clean stems out of a text. A list of clean stems are returned """
    tokenizer = RegexpTokenizer(r'\s+', gaps=True)
    tokens = tokenizer.tokenize(text)
    stems = []  # result
    for token in tokens:
        stem = token.lower()
        stem = stem.strip(string.punctuation)
        if stem:
            if is_number(stem):
                stem = NUMBER_INDICATOR
            elif is_currency(stem):
                stem = CURRENCY_INDICATOR
            elif is_chemical(stem):
                stem = CHEMICAL_INDICATOR
            else:
                stem = stem.strip(string.punctuation)
            if stem and len(stem) >= MIN_SIZE:
                # extract uni-grams
                stems.append(stem)
    del tokens
    return stems

def is_number(str):
    """ Returns true if given string is a number (float or int)"""
    try:
        float(str.replace(",", ""))
        return True
    except ValueError:
        return False

def is_currency(str):
    return str[0] == "$"

def is_chemical(str):
    return str.count("-") > 3

In [16]:
MIN_SIZE = 0
NUMBER_INDICATOR = "number_inidicator"
CURRENCY_INDICATOR = "currency_inidicator"
CHEMICAL_INDICATOR = "chemical_inidicator"


BATCH_SIZE = 10000
# TRAINING_PREPROCESSED_FILES_PREFIX = "/mnt/data2/shalaby/training_docs_sample_%s_data_preprocessed-" % str(SAMPLE_RATIO)
# TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX = "/mnt/data2/shalaby/training_docs_sample_%s_docids_preprocessed-" % str(SAMPLE_RATIO)
# VALIDATION_PREPROCESSED_FILES_PREFIX = "/mnt/data2/shalaby/validation_docs_sample_%s_data_preprocessed-" % str(SAMPLE_RATIO)
# VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX = "/mnt/data2/shalaby/validation_docs_sample_%s_docids_preprocessed-" % str(SAMPLE_RATIO)

TRAINING_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_additional_data_preprocessed"
VALIDATION_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_additional_data_preprocessed"


TRAINING_MERGED_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_merged_data_preprocessed-"
TRAINING_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_merged_docids_preprocessed-"
VALIDATION_MERGED_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_merged_data_preprocessed-"
VALIDATION_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_merged_docids_preprocessed-"

In [17]:
def write_batch(file_name, batch_lines):
    if len(batch_lines):
        with open(file_name, 'w') as batch_file:
            for line in batch_lines:
                batch_file.write((u" ".join(line) + "\n").encode('utf-8'))


In [13]:
%%time
doc_line_positions = dict()
with open(training_file) as f:
    doc_position = f.tell()
    line = f.readline()
    while line:
        if not line.strip(): continue
        doc_id = line[3:11]
        doc_line_positions[doc_id] = doc_position
        doc_position = f.tell()
        line = f.readline()

CPU times: user 5min 16s, sys: 1min 14s, total: 6min 31s
Wall time: 14min 52s


#### Create Training Batch File

In [ ]:
%%time
with open(training_file) as f:
    token_lines = []
    for training_doc in additional_training_docs_list:
        f.seek(doc_line_positions[training_doc])
        line = f.readline()
        (doc_id, text) = eval(line)
        token_lines.append(stemtokenizer(text))
    write_batch(TRAINING_PREPROCESSED_FILES_PREFIX, token_lines)

#### Create Validation Batch File

In [24]:
%%time
with open(training_file) as f:
    token_lines = []
    for validation_doc in additional_validation_docs_list:
        f.seek(doc_line_positions[validation_doc])
        line = f.readline()
        (doc_id, text) = eval(line)
        token_lines.append(stemtokenizer(text))
    write_batch(VALIDATION_PREPROCESSED_FILES_PREFIX, token_lines)

CPU times: user 1.7 s, sys: 24 ms, total: 1.72 s
Wall time: 1.84 s
